In [1]:
# imports
from bs4 import BeautifulSoup
import requests, csv, pandas as pd

In [2]:
# url, request and response
url = 'https://store.steampowered.com/hwsurvey/videocard/'
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.content)


In [4]:
data = soup.findAll('div', id='sub_stats')[0]

In [5]:
#print(data)

In [6]:
# extracting recent 5 months
data1 = data.findAll('div', class_='col_header')[1:6]
months = []
for month in range(len(data1)):
    months.append(data1[month].text)

In [7]:
# data extraction line for gpus
data2 = data.findAll('div', class_='substats_col_left')[8:109]

In [8]:
# data extraction lines for Recent month column %
data3 = data.findAll('div', class_='substats_col_month_last_pct')[8:109]

In [9]:
# data extraction for other months
data4 = data.findAll('div', class_='substats_col_month')[32:]

In [10]:
# creating months-percent dict
numerate = 0
months_percent = {
    months[0] : [], 
    months[1] : [],
    months[2] : [],
    months[3] : [],
    months[4] : []
}
for percent in data4:

    if ('JUN' in percent.text):
        break
    months_percent[months[numerate]].append(percent.text[:-1])
    numerate += 1
    if (numerate == 4):
        numerate = 0

for percent in data3:
    months_percent[months[4]].append(percent.text[:-1])

In [11]:
# Getting list of gpus
gpu_percent_dict = {}
num = 0
for entry in data2:
    temp = [
        months_percent[months[0]][num],
        months_percent[months[1]][num],
        months_percent[months[2]][num],
        months_percent[months[3]][num],
        months_percent[months[4]][num]
    ]
    if ( ('NVIDIA' in entry.text) or ('AMD' in entry.text) or ('Intel' in entry.text) ):
        gpu_percent_dict[entry.text.strip()] = temp
    if ( ('Other' in entry.text)):
        gpu_percent_dict['Other'] = temp

    num += 1

In [12]:
#print(gpu_percent_dict)

In [13]:
# create dataframe
df = pd.DataFrame.from_dict(gpu_percent_dict, orient='index', columns=[
    months[0],
    months[1],
    months[2],
    months[3],
    months[4]])

In [14]:
# Dataframe Reorganization
df['GPU'] = df.index
df = df[['GPU',months[0],months[1],months[2],months[3],months[4]]]
df.reset_index(drop=True)

,GPU,JUN,JUL,AUG,SEP,OCT
0,NVIDIA GeForce GTX 1060,9.88,9.17,10.46,8.64,8.08
1,NVIDIA GeForce GTX 1050 Ti,6.46,6.41,6.14,6.16,5.94
2,NVIDIA GeForce GTX 1650,5.61,5.62,5.36,5.99,5.94
3,NVIDIA GeForce RTX 2060,5.99,5.52,6.87,5.57,5.16
4,NVIDIA GeForce GTX 1050,3.73,3.74,3.35,3.43,3.39
...,...,...,...,...,...,...
96,AMD Radeon HD 7700 Series,0.16,0.17,,0.15,0.16
97,NVIDIA GeForce 840M,0.18,0.17,,,0.15
98,Intel Valleyview Baytrail,0.18,0.18,0.15,0.15,0.15
99,NVIDIA GeForce 920MX,0.17,0.17,,0.15,0.15


In [15]:
# test df methods
len(df)

101

In [16]:
# writing to csv
df.to_csv(path_or_buf='steam_gpu_survey')